# Phase 1: Classification (assign Utility, Application, or Entity Tag)

## 1.1 Create Embeddings

In [ ]:
version = "v_team" # TODO changer
system = "pos" # TODO changer
model_type = "albert" # or ft_codebert, word2vec, albert, codebert, roberta, bert

In [ ]:
from transformers import AutoTokenizer, AutoModel, AlbertTokenizer, AlbertModel, RobertaModel, RobertaTokenizer, BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from utils import load_class_code_from_directory, load_data_from_csv, save_embeddings_to_csv, associate_classes_to_types
from embeddings import generate_embeddings_for_java_code, generate_word_embeddings_for_java_code
import nltk
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec # not used
import gensim.downloader as api

In [ ]:
# Check if CUDA (GPU) is available and if so, set the device to GPU
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)

In [ ]:
# Select the model and tokenizer
if (model_type == "codebert"):
    tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base",force_download=False)
    model = AutoModel.from_pretrained("microsoft/codebert-base",force_download=False)
elif (model_type == "ft_codebert"):
    tokenizer = AutoTokenizer.from_pretrained("./codebert_finetuned",force_download=False)
    model = AutoModel.from_pretrained("./codebert_finetuned",force_download=False)
elif (model_type == "bert"):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") 
    model = BertModel.from_pretrained("bert-base-uncased") 
elif (model_type == "roberta"):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = RobertaModel.from_pretrained("roberta-base")
elif (model_type == "albert"): 
    # pip3 install sentencepiece
    tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
    model = AlbertModel.from_pretrained("albert-base-v2")
elif model_type == "word2vec":
    # Download required NLTK datasets and initialize the lemmatizer
    nltk.download('wordnet')
    word_lemmatizer = WordNetLemmatizer()

    # Load Word2Vec model
    word2vec_model = api.load('word2vec-google-news-300')
else:
    raise NameError("model type not supported")

# Move the model to the GPU if available
model = model.to(device)

In [ ]:
# Labels are 0: Application, 1: Utility, 2: Entity
class_labels = associate_classes_to_types(version, system)
print(class_labels)

In [ ]:
# For each class in class_code, generate embeddings and add to class_embeddings dictionary
class_embeddings = {}
class_code = load_class_code_from_directory(system)
if model_type == "word2vec":
    class_embeddings = {class_name: generate_word_embeddings_for_java_code(code) for class_name, code in class_code.items()}
else:
    class_embeddings = {class_name: generate_embeddings_for_java_code(code, tokenizer, model, device) for class_name, code in class_code.items()}

# Write embeddings to csv file
save_embeddings_to_csv(version, system, model_type, class_embeddings, class_labels)

## 1.2 Train ML models

In [ ]:
filename = f"{version}_{system}_{model_type}_embeddings.csv"
class_names, labels, embeddings = load_data_from_csv(filename)

Xtrain, Xtest, names_train, names_test = train_test_split(embeddings, class_names, test_size=0.3, random_state=0)

ytrain = [labels[class_names.index(name)] for name in names_train]
ytest = [labels[class_names.index(name)] for name in names_test]

Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)

# Ensure that there's at least one instance of the "Utility" label in the training data
if 1 not in ytrain:
    utility_index = labels.index(1)
    Xtrain = np.append(Xtrain, [embeddings[utility_index]], axis=0)
    ytrain.append(1)

print(Xtrain)
print(len(Xtest))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
def generate_classification_report(y_true, y_pred):
    # Identify unique labels in both true labels and predictions
    unique_labels = np.unique(np.concatenate((y_true, y_pred)))

    # Map unique labels to their corresponding names
    label_names_map = {-1: "None", 0: "Application", 1: "Utility", 2: "Entity"}
    dynamic_label_names = [label_names_map[label] for label in unique_labels]

    # Generate and print the classification report
    print(classification_report(y_true, y_pred, target_names=dynamic_label_names))

## Decision Tree

In [ ]:
decision_tree_classifier = DecisionTreeClassifier(max_depth=2).fit(Xtrain, ytrain)
decision_tree_predictions = decision_tree_classifier.predict(Xtest)
decision_tree_accuracy = accuracy_score(ytest, decision_tree_predictions)
decision_tree_confusion_matrix = confusion_matrix(ytest, decision_tree_predictions)
print(decision_tree_accuracy)
print(decision_tree_confusion_matrix)
generate_classification_report(ytest, decision_tree_predictions)

## SVM

In [ ]:
svm_classifier = SVC(kernel='linear', C=2).fit(Xtrain, ytrain)
svm_predictions = svm_classifier.predict(Xtest)
svm_accuracy = accuracy_score(ytest, svm_predictions)
svm_confusion_matrix = confusion_matrix(ytest, svm_predictions)
print(svm_accuracy)
print(svm_confusion_matrix)
generate_classification_report(ytest, svm_predictions)

## KNN

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5).fit(Xtrain, ytrain)
knn_predictions = knn_classifier.predict(Xtest)
knn_accuracy = accuracy_score(ytest, knn_predictions)
knn_confusion_matrix = confusion_matrix(ytest, knn_predictions)
print(knn_accuracy)
print(knn_confusion_matrix)
generate_classification_report(ytest, knn_predictions)

## LogisticRegression

In [ ]:
logistic_regression_classifier = LogisticRegression(random_state=0).fit(Xtrain, ytrain)
logistic_regression_predictions = logistic_regression_classifier.predict(Xtest)
logistic_regression_accuracy = accuracy_score(ytest, logistic_regression_predictions)
logistic_regression_confusion_matrix = confusion_matrix(ytest, logistic_regression_predictions)
print(logistic_regression_accuracy)
print(logistic_regression_confusion_matrix)
generate_classification_report(ytest, logistic_regression_predictions)

## Gaussian NB

In [ ]:
naive_bayes_classifier = GaussianNB().fit(Xtrain, ytrain)
naive_bayes_predictions = naive_bayes_classifier.predict(Xtest)
naive_bayes_accuracy = accuracy_score(ytest, naive_bayes_predictions)
naive_bayes_confusion_matrix = confusion_matrix(ytest, naive_bayes_predictions)
print(naive_bayes_accuracy)
print(naive_bayes_confusion_matrix)
generate_classification_report(ytest, naive_bayes_predictions)